## Iniciar Spark Session

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext
sc

24/06/17 03:39:08 WARN Utils: Your hostname, MacBook-Pro-de-Sebastian-1028.local resolves to a loopback address: 127.0.0.1; using 192.168.1.200 instead (on interface en0)
24/06/17 03:39:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/17 03:39:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=pyspark-shell>

## Instanciar BBD Neo4j

In [2]:
import neo4j
from neo4j import GraphDatabase
import itertools

In [3]:
URI = "neo4j+s://a3d230a8.databases.neo4j.io"
AUTH = ("neo4j", "PDOH3crDmCKi-BEnScXmynkqDkuc2bZGmv5WgHgG6lQ")

driver = GraphDatabase.driver(URI, auth=AUTH)
with driver.session() as session:
    try:
        session.run("RETURN 1")
        print("Connection to Neo4j established successfully!")
    except Exception as e:
        print(f"Failed to connect to Neo4j: {e}")

Connection to Neo4j established successfully!


In [4]:
def create_graph(tx, graph_data):
    # Create nodes
    nodes = set()
    for start_node, relationship_type, end_node in graph_data:
        nodes.add(start_node)
        nodes.add(end_node)

    for node in nodes:
        tx.run("MERGE (n:Node {id: $id})", id=node)

    # Create relationships
    for start_node, relationship_type, end_node in graph_data:
        tx.run(
            """
            MATCH (a:Node {id: $start_id})
            MATCH (b:Node {id: $end_id})
            MERGE (a)-[r:RELATIONSHIP {type: $type}]->(b)
            """,
            start_id=start_node,
            end_id=end_node,
            type=relationship_type,
        )

In [5]:
# Example graph data

graph_data = [
    (1, 11, 2),
    (1, 11, 3),
    (2, 11, 3),
    (3, 11, 2),
    (3, 11, 4),
    (4, 11, 1),
    (4, 11, 2),
    (4, 11, 3),
    (4, 12, 5),
    (5, 12, 1),
    (5, 12, 2),
    (5, 12, 6),
]

In [6]:
with driver.session() as session:
    try:
        session.execute_write(create_graph, graph_data)
        print("Graph instantiated successfully!")
    except Exception as e:
        print(f"Failed to instantiate the graph: {e}")

Graph instantiated successfully!


## Problema 1

Implementa una función que reciba un grafo en Neo4j y genere una RDD con las aristas de ese grafo.


In [7]:
from collections import namedtuple

Edge = namedtuple("Edge", ["n1", "R", "n2"])

In [8]:
# Get edges from example graph database

example_graph_query = """
    MATCH (a:Node)-[r:RELATIONSHIP]->(b:Node)
    RETURN a.id AS start_node, r.type AS relationship_type, b.id AS end_node
    """

query = """
    MATCH (a:Paper)-[r:CITES]->(b:Paper)
    RETURN a.id AS start_node, b.id AS end_node
    """

with driver.session() as session:
    result = session.run(query)
    edges = [
        Edge(record["start_node"], "cites", record["end_node"]) for record in result
    ]
    # result = session.run(example_graph_query)
    # edges = [Edge(record["start_node"], record["relationship_type"], record["end_node"]) for record in result]

# Create rdd from edges

graph_rdd = sc.parallelize(edges)

graph_rdd.collect()

[Edge(n1=31349, R='cites', n2=31336),
 Edge(n1=686532, R='cites', n2=31336),
 Edge(n1=1129442, R='cites', n2=31336),
 Edge(n1=1107312, R='cites', n2=13195),
 Edge(n1=1120731, R='cites', n2=13195),
 Edge(n1=755217, R='cites', n2=13195),
 Edge(n1=1105116, R='cites', n2=37879),
 Edge(n1=686532, R='cites', n2=31349),
 Edge(n1=137849, R='cites', n2=109323),
 Edge(n1=154134, R='cites', n2=217139),
 Edge(n1=31336, R='cites', n2=31353),
 Edge(n1=31349, R='cites', n2=31353),
 Edge(n1=1152272, R='cites', n2=31353),
 Edge(n1=1124844, R='cites', n2=31353),
 Edge(n1=1135746, R='cites', n2=31353),
 Edge(n1=10531, R='cites', n2=31353),
 Edge(n1=194617, R='cites', n2=31353),
 Edge(n1=286562, R='cites', n2=31353),
 Edge(n1=1123576, R='cites', n2=31353),
 Edge(n1=1107567, R='cites', n2=31353),
 Edge(n1=686532, R='cites', n2=31353),
 Edge(n1=1063773, R='cites', n2=31353),
 Edge(n1=686559, R='cites', n2=31353),
 Edge(n1=1129442, R='cites', n2=31353),
 Edge(n1=31927, R='cites', n2=31353),
 Edge(n1=1152162,

## Problema 2

* Implementa un programa en PySpark que entregue todos los triángulos (como tuplas de tres nodos) en el grafo usando b3 reducers, donde b es un parámetro. Para esta primera parte puedes asumir que tu grafo solo usa una etiqueta de arista (en el grafo de prueba, esa etiqueta corresponde al numero 11).

### Fase de Map

* Generamos pares de (llave, valor) para las aristas del grafo.

In [225]:
# Map procedure
def hash_node(node_id: int, b: int) -> int:
    return hash(node_id) % b


def generate_n_keys(edge, b, n):
    hashed_n1 = hash_node(edge.n1, b)
    hashed_n2 = hash_node(edge.n2, b)

    combinations = itertools.product(range(b), repeat=n - 2)

    return (((hashed_n1, hashed_n2) + comb, edge) for comb in combinations)

### Fase de Reduce

* Agrupamos llaves según si coinciden en términos de un shift de carrusel: (b1, b2, b3) = (b2, b3, b1) = (b3, b1, b2)
* Detectamos ciclos del tamaño deseado para cada reducer mediante DFS y retornamos los nodos asociados

In [226]:
def generate_shifts(key):
    n = len(key)
    gen = [tuple(key[i:] + key[:i]) for i in range(n)]
    return gen


def map_to_shifts(key_edge_tuple):
    key, edge_value = key_edge_tuple

    return [((shifted_key), edge_value) for shifted_key in generate_shifts(key)]

In [425]:
def create_graph(edges):
    graph = {}

    for edge in edges:
        start = edge.n1
        end = edge.n2
        if start not in graph:
            graph[start] = []
        graph[start].append(end)

    return graph


def dfs_iterative(graph, start_node, n):
    stack = [(start_node, 0, [start_node])]

    while stack:
        node, path_length, path = stack.pop()

        if path_length == n:
            if node == start_node:
                return True, tuple(set(path))

            continue

        if node in graph:
            for neighbor in graph[node]:
                new_path = path + [neighbor]
                stack.append((neighbor, path_length + 1, new_path))

    return False, ()


def has_cycle_of_length_n(edges, n):
    graph = create_graph(edges)

    for node in graph:
        cycle_detected, cycle_nodes = dfs_iterative(graph, node, n)

        if cycle_detected:
            return cycle_detected, cycle_nodes

    return False, ()

In [228]:
# Example usage:
edges = [
    Edge(1, "cites", 2),
    Edge(2, "cites", 3),
    Edge(3, "cites", 4),
    Edge(4, "cites", 1),
    Edge(2, "cites", 4),
    Edge(4, "cites", 5),
]
n = 3  # Length of the cycle we want to detect


cycle_detected, cycle_nodes = has_cycle_of_length_n(edges, n)
if cycle_detected:
    print(f"Cycle of length {n} detected: {cycle_nodes}")
else:
    print(f"No cycle of length {n} found.")

Cycle of length 3 detected: (1, 2, 4)


### Pipeline de datos

In [219]:
b = 5  # Number of buckets
n = 3  # Length of the cycle we want to detect

mapped_graph_rdd = graph_rdd.flatMap(lambda edge: generate_n_keys(edge, b, n))

aggregated_rdd = mapped_graph_rdd.flatMap(map_to_shifts).groupByKey().mapValues(list)

cycles_rdd = aggregated_rdd.map(
    lambda key_edge_pair: (key_edge_pair[0], has_cycle_of_length_n(key_edge_pair[1], n))
)

nodes_rdd = cycles_rdd.filter(lambda x: x[1][0]).map(lambda x: (x[0], x[1][1]))

queried_triangles = nodes_rdd.values().distinct()

In [220]:
queried_triangles.collect()

[(51180, 12350, 67415),
 (20178, 91852, 64271),
 (12195, 51180, 12350),
 (273152, 35, 210871),
 (87482, 34266, 34263),
 (561568, 561613, 153063),
 (753264, 753070, 753047),
 (12576, 56112, 83725),
 (3192, 5086, 3191),
 (126920, 126927, 103543),
 (215912, 194617, 31932),
 (119761, 284025, 143801),
 (643221, 642894, 643239),
 (126920, 645897, 126927),
 (60169, 399370, 60159),
 (31769, 67245, 67246),
 (184918, 330148, 23502)]

## Problema 3

* Asume ahora que recibes un subgrafo como tres arreglos: un arreglo A con las variables, otro L con los tipos de aristas, y una matriz M de tamaño |A| × |L| × |A| que tiene un uno en la posicion (x, R, y) si y solo si (x, R, y) es una arista de tu subgrafo. 
* Implementa un programa en PySpark que reciba un patrón que tiene solo variables, y exactamente cuatro variables, y entregue todos los matches de ese patrón (como tuplas de 4 nodos) en el grafo usando b4 reducers, donde b es un parámetro.

### Map

Para este caso el Map se hara solamente para los casos las combinaciones donde las relaciones son especificadas en el patrón.

Con esto reducimos el espacio de búsqueda y incorporamos las relaciones en el proceso de búsqueda.

In [519]:
# Map procedure
def hash_node(node_id: int, b: int) -> int:
    return hash(node_id) % b


def insert_hashes_recursively(comb, hashes, positions, idx=0):
    if not positions:
        return comb[:idx] + hashes + comb[idx:]

    pos = positions[0]
    next_comb = comb[:pos] + hashes[:1] + comb[pos:]
    return insert_hashes_recursively(
        next_comb, hashes[1:], [p + 1 for p in positions[1:]], idx
    )


def generate_keys(edge, b, rel_order):
    hashed_n1 = hash_node(edge.n1, b)
    hashed_n2 = hash_node(edge.n2, b)
    edge_rel = edge.R

    keys = []
    n = len(rel_order)
    combinations = itertools.product(range(b), repeat=n - 2)

    # Generar llaves según el orden de las relaciones
    for comb in combinations:
        if edge_rel == rel_order[0]:
            keys.append(((hashed_n1, hashed_n2) + comb, edge))
        if edge_rel == rel_order[1]:
            keys.append(((comb[0], hashed_n1, hashed_n2) + comb[1:], edge))

        if n == 3 and edge_rel == rel_order[2]:
            keys.append(((hashed_n2, comb[0], hashed_n1) + comb[1:], edge))

        if n == 4:
            if edge_rel == rel_order[2]:
                keys.append(((comb[0], comb[1], hashed_n1, hashed_n2), edge))
            if edge_rel == rel_order[3]:
                keys.append(((hashed_n2, comb[0], comb[1], hashed_n1), edge))

    return keys

### Reduce

Para este ejemplo usamos el dado en parte del enunciado dado que los papersm unicamente son citas, no tienen tipo de relaciones.

In [520]:
data = [
    Edge(1, 11, 2),
    Edge(1, 11, 3),
    Edge(2, 11, 3),
    Edge(3, 11, 2),
    Edge(3, 11, 4),
    Edge(4, 11, 1),
    Edge(4, 11, 2),
    Edge(4, 11, 3),
    Edge(4, 12, 5),
    Edge(5, 12, 1),
    Edge(5, 12, 2),
    Edge(5, 12, 6),
]
graph_rdd = sc.parallelize(data)

graph_rdd.collect()

[Edge(n1=1, R=11, n2=2),
 Edge(n1=1, R=11, n2=3),
 Edge(n1=2, R=11, n2=3),
 Edge(n1=3, R=11, n2=2),
 Edge(n1=3, R=11, n2=4),
 Edge(n1=4, R=11, n2=1),
 Edge(n1=4, R=11, n2=2),
 Edge(n1=4, R=11, n2=3),
 Edge(n1=4, R=12, n2=5),
 Edge(n1=5, R=12, n2=1),
 Edge(n1=5, R=12, n2=2),
 Edge(n1=5, R=12, n2=6)]

### Adapt DFS

Para este caso, lo que hicimos fue adaptar el anterior algoritmo de DFS, para asi incoporar las relaciones y que ademas de buscar, tambien asegurarnos el grafo sigue cierto orden sequencial.

In [521]:
def create_graph(edges):
    graph = {}

    for edge in edges:
        start = edge.n1
        end = edge.n2
        relation = edge.R
        if start not in graph:
            graph[start] = []
        graph[start].append((end, relation))

    return graph


def dfs__relation_iterative(graph, start_node, n, rel_order):
    stack = [(start_node, 0, [start_node], [])]
    visited_paths = set()

    while stack:
        node, path_length, path, rels = stack.pop()

        if path_length == n:
            if node == start_node and rels == rel_order:
                return True, tuple(set(path))
            continue

        if node in graph:
            for neighbor, relation in graph[node]:
                new_path = path + [neighbor]
                new_rels = rels + [relation]
                new_path_tuple = (node, relation, neighbor)

                # Here we add memory to dont have repeated paths, so 2 -> 3, 3 -> 2, is a cycle of length 2, not 2n
                if new_path_tuple not in visited_paths:
                    visited_paths.add((node, relation, neighbor))
                    stack.append((neighbor, path_length + 1, new_path, new_rels))
                    visited_paths.add(new_path_tuple)

    return False, ()


def has_cycle_of_length_n_with_order(edges, n, rel_order):
    graph = create_graph(edges)

    for node in graph:
        cycle_detected, cycle_nodes = dfs__relation_iterative(graph, node, n, rel_order)
        if cycle_detected:
            return cycle_detected, cycle_nodes

    return False, ()

In [522]:
b = 4  # Number of buckets

R_1 = 11
R_2 = 11
R_3 = 11
R_4 = 11
relations = [R_1, R_2, R_3, R_4]
n = len(relations)  # Length of the cycle we want to detect

mapped_graph_rdd = graph_rdd.flatMap(lambda edge: generate_keys(edge, b, relations))
mapped_graph_rdd.count()
aggregated_rdd = mapped_graph_rdd.groupByKey().mapValues(list)

cycles_rdd = aggregated_rdd.map(
    lambda key_edge_pair: (
        key_edge_pair[0],
        has_cycle_of_length_n_with_order(key_edge_pair[1], n, relations),
    )
)


nodes_rdd = cycles_rdd.filter(lambda x: x[1][0]).map(lambda x: (x[0], x[1][1]))

queried_triangles = nodes_rdd.values().distinct()

queried_triangles.collect()

[((3, 0, 0, 0), Edge(n1=3, R=11, n2=4)), ((0, 3, 0, 0), Edge(n1=3, R=11, n2=4)), ((0, 3, 0, 0), Edge(n1=3, R=11, n2=4)), ((0, 0, 3, 0), Edge(n1=3, R=11, n2=4)), ((0, 0, 0, 3), Edge(n1=3, R=11, n2=4)), ((3, 0, 0, 1), Edge(n1=3, R=11, n2=4)), ((0, 3, 0, 1), Edge(n1=3, R=11, n2=4)), ((0, 3, 1, 0), Edge(n1=3, R=11, n2=4)), ((0, 1, 3, 0), Edge(n1=3, R=11, n2=4)), ((0, 0, 1, 3), Edge(n1=3, R=11, n2=4)), ((3, 0, 0, 2), Edge(n1=3, R=11, n2=4)), ((0, 3, 0, 2), Edge(n1=3, R=11, n2=4)), ((0, 3, 2, 0), Edge(n1=3, R=11, n2=4)), ((0, 2, 3, 0), Edge(n1=3, R=11, n2=4)), ((0, 0, 2, 3), Edge(n1=3, R=11, n2=4)), ((3, 0, 0, 3), Edge(n1=3, R=11, n2=4)), ((0, 3, 0, 3), Edge(n1=3, R=11, n2=4)), ((0, 3, 3, 0), Edge(n1=3, R=11, n2=4)), ((0, 3, 3, 0), Edge(n1=3, R=11, n2=4)), ((0, 0, 3, 3), Edge(n1=3, R=11, n2=4)), ((3, 0, 1, 0), Edge(n1=3, R=11, n2=4)), ((1, 3, 0, 0), Edge(n1=3, R=11, n2=4)), ((1, 3, 0, 0), Edge(n1=3, R=11, n2=4)), ((1, 0, 3, 0), Edge(n1=3, R=11, n2=4)), ((0, 1, 0, 3), Edge(n1=3, R=11, n2=4)),

[(1, 2, 3, 4)]